# TensorFlow Data loading

In [1]:
import tensorflow as tf
import os
import numpy as np
import random

## Using inbuilt loader

In [2]:
data_dir = '/Users/georgebrockman/code/georgebrockman/Autoenhance.ai/RoomDetection/images/training_data/'

In [3]:
IMG_WIDTH, IMG_HEIGHT = 224, 224
IMG_SIZE = IMG_WIDTH, IMG_HEIGHT
batch_size = 32

In [4]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(IMG_WIDTH, IMG_HEIGHT),
  batch_size=batch_size)

Found 27666 files belonging to 9 classes.
Using 22133 files for training.


In [5]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(IMG_WIDTH, IMG_HEIGHT),
  batch_size=batch_size)

Found 27666 files belonging to 9 classes.
Using 5533 files for validation.


In [6]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

### Import the base model

In [7]:
# instantiate the MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                             include_top=False,
                                             weights='imagenet')

In [8]:
image_batch, label_batch = next(iter(train_ds)) # the next iteration in the dataset, so the first image
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(32, 7, 7, 1280)


In [9]:
# freeze the convolutional base
base_model.trainable=False

In [10]:
# convert the features to a single 1280-element vector per image
global_av_layer = tf.keras.layers.GlobalAveragePooling2D() # averages over a 5x5 spatial 
feature_batch_av = global_av_layer(feature_batch)
print(feature_batch_av.shape)

(32, 1280)


In [11]:
# apply a dense layer to convert these features into a single prediction per image
# no activation needed as the prediction will be treated as a logit (mapping of probabilities to Real Numbers)

pred_layer = tf.keras.layers.Dense(1)
pred_batch = pred_layer(feature_batch_av)
pred_batch.shape

TensorShape([32, 1])

### Build the model

In [12]:
# augmentate the data 

data_aug = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(.5, .2),
    tf.keras.layers.experimental.preprocessing.RandomContrast((0.5,0.5), seed=1),
])

# rescale the pixel values to match the expected values of the MobileNetV2 model

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [13]:
# chain together data augmentation, rescaling, base_model and feature extractor layers useing the Keras Functional API

inputs = tf.keras.Input(shape=(224,224,3)) # image size and channels
# data augmentation layer
x = data_aug(inputs)
# preprocess, feed x into and reassign variable
x = preprocess_input(x)
# basemodel, set training =False for the BN layer
x = base_model(x, training=False)
# feature extraction
x = global_av_layer(x)
# add a dropout layer 
x = tf.keras.layers.Dropout(0.2)(x)

outputs = pred_layer(x)
model = tf.keras.Model(inputs, outputs)

### Compile the model

In [14]:
METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
]


In [23]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              # Only two linear outputs so use BinaryCrossentropy and logits =True
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

### Train the model

In [ ]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(outputs),
                                                 outputs)

In [24]:
initial_epochs = 10
history = model.fit(train_ds,
                    epochs=initial_epochs,
                    validation_data=val_ds,)

Epoch 1/10
692/692 [==============================] - 587s 849ms/step - loss: -131.0499 - accuracy: 0.2013 - val_loss: -153.7231 - val_accuracy: 0.1984
Epoch 2/10
692/692 [==============================] - 515s 744ms/step - loss: -235.4192 - accuracy: 0.2013 - val_loss: -240.4066 - val_accuracy: 0.1984
Epoch 3/10
692/692 [==============================] - 516s 746ms/step - loss: -339.4034 - accuracy: 0.2013 - val_loss: -326.8677 - val_accuracy: 0.1984
Epoch 4/10
692/692 [==============================] - 520s 751ms/step - loss: -441.2041 - accuracy: 0.2013 - val_loss: -413.2120 - val_accuracy: 0.1984
Epoch 5/10
692/692 [==============================] - 531s 768ms/step - loss: -545.3388 - accuracy: 0.2013 - val_loss: -499.2839 - val_accuracy: 0.1984
Epoch 6/10
692/692 [==============================] - 516s 746ms/step - loss: -650.7616 - accuracy: 0.2013 - val_loss: -585.6879 - val_accuracy: 0.1984
Epoch 7/10
692/692 [==============================] - 538s 778ms/step - loss: -755.1579 

In [ ]:
# balance the classes to help with the accuracy

# Using custom built loading function to build the model (with no valiadtion set)

In [2]:
def dataset_classifcation(path, resize_h, resize_w, train=True, limit=None):
    
    # list all paths to data classes except DS_Store
    class_folders = [f for f in sorted(os.listdir(path)) if not f.startswith('.')]
    # load images
    images = []
    classes = []
    for i, c in enumerate(class_folders):
        #images_per_class = sorted(os.path.join(path, c))
        images_per_class = [f for f in sorted(os.listdir(os.path.join(path, c))) if 'jpg' in f]
        image_class = np.zeros(len(class_folders))
        image_class[i] = 1
        
        for image_per_class in images_per_class:
            images.append(os.path.join(path, c, image_per_class))
            # the index will be the class label
            classes.append(image_class)
    
    random.seed(10)
    images_shuffle = random.sample(images, len(images))
    classes_shuffle = random.sample(classes, len(classes))
    train_test_split = 0.1
    number_of_test = int(len(images) * train_test_split)
    if train == False:
        images = images_shuffle[0:number_of_test]
        classes = classes_shuffle[0:number_of_test]
    else:
        images = images_shuffle[number_of_test:len(images)]
        classes = classes_shuffle[number_of_test:len(images)]
    
    images_tf = tf.data.Dataset.from_tensor_slices(images)
    classes_tf = tf.data.Dataset.from_tensor_slices(classes)
    # put two arrays together so that each image has its classifying label 
    dataset = tf.data.Dataset.zip((images_tf, classes_tf))
    
    @tf.function
    def read_images(image_path, class_type, mirrored=False):
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image)

        h, w, c = image.shape
        if not (h == resize_h and w == resize_w):
            image = tf.image.resize(
            image, [resize_h, resize_w], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
            # set all images shape to RGB
            image.set_shape((224, 224, 3))
#             print(image.shape)
    
    
        # change DType of image to float32
        image = tf.cast(image, tf.float32)
        class_type = tf.cast(class_type, tf.float32)
        
        # normalise the image pixels
        image = (image / 255.0)

        return image, class_type

    dataset = dataset.map(
        read_images,
        num_parallel_calls=tf.data.experimental.AUTOTUNE,
        deterministic=False)

    return dataset, len(class_folders)

In [3]:
# import the data 

path = '/Users/georgebrockman/code/georgebrockman/Autoenhance.ai/RoomDetection/images/training_data/'
train_dataset, num_classes = dataset_classifcation(path, 224, 224)
test_dataset, num_classes = dataset_classifcation(path, 224, 224, train=False)

In [4]:
train_dataset

<ParallelMapDataset shapes: ((224, 224, 3), (9,)), types: (tf.float32, tf.float32)>

In [5]:
IMG_WIDTH, IMG_HEIGHT = 224, 224
IMG_SIZE = IMG_WIDTH, IMG_HEIGHT
batch_size = 32

In [6]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(1000).batch(32).prefetch(buffer_size=AUTOTUNE)

In [7]:
test_dataset = test_dataset.cache().batch(32).prefetch(buffer_size=AUTOTUNE)

In [8]:
# import the base model
# instantiate the MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                             include_top=False,
                                             weights='imagenet')

In [9]:
image_batch, label_batch = next(iter(train_dataset))# the next iteration in the dataset, so the first image
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(32, 7, 7, 1280)


In [10]:
# freeze the convolutional base
base_model.trainable=False

In [11]:
# convert the features to a single 1280-element vector per image
global_av_layer = tf.keras.layers.GlobalAveragePooling2D() # averages over a 5x5 spatial 
feature_batch_av = global_av_layer(feature_batch)
print(feature_batch_av.shape)

(32, 1280)


In [12]:
# apply a dense layer to convert these features into a single prediction per image
# no activation needed as the prediction will be treated as a logit (mapping of probabilities to Real Numbers)

pred_layer = tf.keras.layers.Dense(num_classes, activation='softmax')
pred_batch = pred_layer(feature_batch_av)
pred_batch.shape

TensorShape([32, 9])

## Build the model volume 2

In [13]:
# augmentate the data 

data_aug = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.25),
    tf.keras.layers.experimental.preprocessing.RandomZoom(.5, .2),
    
])

# rescale the pixel values to match the expected values of the MobileNetV2 model

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [14]:
# chain together data augmentation, rescaling, base_model and feature extractor layers useing the Keras Functional API

inputs = tf.keras.Input(shape=(224,224,3)) # image size and channels
# data augmentation layer
x = data_aug(inputs)
# preprocess, feed x into and reassign variable
x = preprocess_input(x)
# basemodel, set training =False for the BN layer
x = base_model(x, training=False)
# feature extraction
x = global_av_layer(x)
# add a dropout layer 
x = tf.keras.layers.Dropout(0.2)(x)

outputs = pred_layer(x)
model = tf.keras.Model(inputs, outputs)

In [15]:
# compile model again

In [19]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              # Only two linear outputs so use BinaryCrossentropy and logits =True
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [20]:
checkpoint_path = "/Users/georgebrockman/code/georgebrockman/Autoenhance.ai/RoomDetection/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [22]:
initial_epochs = 10
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    callbacks=[cp_callback],
                    validation_data= test_dataset)
model.save('room_detection.h5')

Epoch 1/10
692/692 [==============================] - ETA: 0s - loss: 2.1399 - accuracy: 0.1903
Epoch 00001: saving model to /Users/georgebrockman/code/georgebrockman/Autoenhance.ai/RoomDetection/cp.ckpt
692/692 [==============================] - 488s 705ms/step - loss: 2.1399 - accuracy: 0.1903 - val_loss: 2.1346 - val_accuracy: 0.2087
Epoch 2/10
692/692 [==============================] - ETA: 0s - loss: 2.1392 - accuracy: 0.1959
Epoch 00002: saving model to /Users/georgebrockman/code/georgebrockman/Autoenhance.ai/RoomDetection/cp.ckpt
692/692 [==============================] - 482s 696ms/step - loss: 2.1392 - accuracy: 0.1959 - val_loss: 2.1348 - val_accuracy: 0.2087
Epoch 3/10
692/692 [==============================] - ETA: 0s - loss: 2.1393 - accuracy: 0.1896
Epoch 00003: saving model to /Users/georgebrockman/code/georgebrockman/Autoenhance.ai/RoomDetection/cp.ckpt
692/692 [==============================] - 560s 810ms/step - loss: 2.1393 - accuracy: 0.1896 - val_loss: 2.1340 - val_

## Fine Tuning

In [23]:
# unfreeze the top layers of the model

base_model.trainable = True

In [24]:
# show how many layers are in the basemodel
print("Number of layers in the base model: ", len(base_model.layers))

Number of layers in the base model:  155


In [25]:
# fine tune for this number onwards
fine_tune_at = 110

# freeze all the layers before the tuning - this can be done with a for loop and slicing
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [26]:
# compile 

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              # Only two linear outputs so use BinaryCrossentropy and logits =True
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [27]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 224, 224, 3)       0         
_________________________________________________________________
tf_op_layer_RealDiv (TensorF [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf_op_layer_Sub (TensorFlowO [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)             

In [30]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         # initial epoch is the last one in the original training, use index -1
                         initial_epoch=history.epoch[-1],
                         validation_data=test_dataset)
model.save('room_detection.h5')

Epoch 10/20
692/692 [==============================] - 611s 883ms/step - loss: 2.1408 - accuracy: 0.1917 - val_loss: 2.1397 - val_accuracy: 0.2087
Epoch 11/20
692/692 [==============================] - 613s 886ms/step - loss: 2.1403 - accuracy: 0.1907 - val_loss: 2.1371 - val_accuracy: 0.2087
Epoch 12/20
692/692 [==============================] - 673s 973ms/step - loss: 2.1405 - accuracy: 0.1937 - val_loss: 2.1348 - val_accuracy: 0.2087
Epoch 13/20
692/692 [==============================] - 618s 893ms/step - loss: 2.1392 - accuracy: 0.1921 - val_loss: 2.1343 - val_accuracy: 0.2087
Epoch 14/20
692/692 [==============================] - 616s 890ms/step - loss: 2.1391 - accuracy: 0.1919 - val_loss: 2.1343 - val_accuracy: 0.2087
Epoch 15/20
692/692 [==============================] - 616s 890ms/step - loss: 2.1401 - accuracy: 0.1962 - val_loss: 2.1339 - val_accuracy: 0.2087
Epoch 16/20
692/692 [==============================] - 607s 877ms/step - loss: 2.1411 - accuracy: 0.1908 - val_loss: 2

In [ ]:
# decrease learning rate
# play around with different layers. 
